In [36]:
import os
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [37]:
CSV_PATH = './submissions/'
ANNOTATION_PATH = '../../dataset/test.json'
RESULT_PATH = './submission_ensemble.csv'
IOU_TREASHOLD = 0.55

In [38]:
submission_files = [os.path.join(CSV_PATH,path) for path in os.listdir(CSV_PATH) if path.endswith('.csv')]
submission_df = [pd.read_csv(file) for file in submission_files]
image_ids = submission_df[0]['image_id'].tolist()
coco = COCO(ANNOTATION_PATH)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [39]:
prediction_strings = []
file_names = []
# ensemble 시 설정할 iou threshold 이 부분을 바꿔가며 대회 metric에 알맞게 적용해봐요!

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
#     각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
#     예측 box가 있다면 이를 ensemble 수행
    if len(boxes_list):
        boxes, scores, labels = nms(boxes_list, scores_list, labels_list, iou_thr=IOU_TREASHOLD)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(label) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [40]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(RESULT_PATH)

submission.head()

,PredictionString,image_id
0,0 0.47424603 488.09274 591.85626 548.1791 657....,test/0000.jpg
1,0 0.7784516 636.91077 662.93365 679.52625 821....,test/0001.jpg
2,0 0.99382025 877.7668 466.15436 1024.0 637.949...,test/0002.jpg
3,0 0.31491753 0.0 47.953247 331.13995 394.2157 ...,test/0003.jpg
4,0 0.44982508 318.554 335.91107 864.6084 772.20...,test/0004.jpg


###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
